In [1]:
# dataframe.shape
# 행 개수, 열 개수
#
#  dataframe.shape[0] # 행 개수
# dataframe.shape[1] # 열 개수

In [ ]:
from google.colab import drive
drive.mount('/gdrive') # google drive 연결
# %cd /gdrive/MyDrive/ ~~~ path 지정

Mounted at /gdrive


In [3]:
%cd /gdrive/MyDrive/수업 정리/재무금융/Corporate Finance/rawdata

/gdrive/.shortcut-targets-by-id/1TW2gBTrdYap0MMMBfQEUFyBjzqW5Ei0G/rawdata


In [2]:
import pandas as pd
import numpy as np

MAX_YEAR = 5.0

In [3]:
import os

os.listdir(".")

['data.pkl',
 'dataguide',
 'main copy.ipynb',
 'main.ipynb',
 'result_freq.csv',
 'result_freq1.csv',
 'result_freq1_3y.csv',
 'result_freq_3y.csv',
 'result_temp.csv',
 'result_temp_3y.csv',
 'workingCap.xlsx',
 '산업분류코드.xlsx']

In [2]:
import os

for f in os.listdir("./dataguide"):
  print(f)
  print(pd.ExcelFile("./dataguide/" + f).sheet_names)

김도연1.xlsx
['op_earn', 'depre_amort', 'depre', 'age', 'n_acq_intangi']
총자산.xlsx
['asset', 'exchange', 'ind_code']


In [3]:
dfs = {
    "ind_code": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="ind_code"),
    "op_earn": pd.read_excel("./dataguide/김도연1.xlsx", sheet_name="op_earn"),
    "depre_amort": pd.read_excel("./dataguide/김도연1.xlsx", sheet_name="depre_amort"),
    "asset": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="asset"),
    "exchange": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="exchange")
}

In [ ]:
# axis = 0 : 가로(행에 대하여) 아마도?
# axis = 1 : 세로(열에 대하여)

In [4]:
rows_to_drop = list(range(0, 13)) # 0 ~ 12

for k in dfs.keys():
  dfs[k].columns = dfs[k].iloc[7] # firmcode line을 column명으로 지정
  dfs[k] = dfs[k].drop(rows_to_drop, axis=0)
  dfs[k] = dfs[k].set_index("Symbol").reset_index().melt(
        id_vars=["Symbol"],
        var_name="firmcode",
        value_name=k,
    ).rename(columns={"Symbol": "date"})

C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the

In [7]:
import pickle

with open("data.pkl", "wb") as f:
  pickle.dump(dfs, f)

In [1]:
import pickle
with open("data.pkl", "rb") as f:
  dfs = pickle.load(f)

In [3]:
for k in dfs.keys():
  print(dfs[k].shape)

(377400, 3)
(377400, 3)
(377400, 3)
(377400, 3)
(377400, 3)


In [4]:
dfs["op_earn"].isna().sum()

date             0
firmcode         0
op_earn     150267
dtype: int64

In [5]:
result = dfs["asset"][["date", "firmcode"]]
for c in dfs.keys():
    result = pd.merge(result, dfs[c], on=["date", "firmcode"], how="left")
result["ebitda"] = result["op_earn"] + result["depre_amort"]
print(result[[
    "ebitda", "op_earn", "depre_amort"
]].isna().sum())
result = result.drop(["op_earn", "depre_amort"], axis=1)
result["month"] = result["date"].dt.month
result["year"] = result["date"].dt.year
result.loc[result["asset"] == 0, "asset"] = None
result.loc[result["ebitda"] == 0, "ebitda"] = None
result

ebitda         185032
op_earn        150267
depre_amort    176877
dtype: int64


,date,firmcode,ind_code,asset,exchange,ebitda,month,year
0,2000-03-31,A005930,C,26895046483000,유가증권시장,7435154660000,3,2000
1,2000-06-30,A005930,C,26895046483000,유가증권시장,7435154660000,6,2000
2,2000-09-30,A005930,C,26895046483000,유가증권시장,7435154660000,9,2000
3,2000-12-31,A005930,C,26895046483000,유가증권시장,7435154660000,12,2000
4,2001-03-31,A005930,C,27919406342000,유가증권시장,2295333697000,3,2001
...,...,...,...,...,...,...,...,...
377395,2023-12-31,A950180,J,NaN,코스닥,NaN,12,2023
377396,2024-03-31,A950180,J,NaN,코스닥,NaN,3,2024
377397,2024-06-30,A950180,J,NaN,코스닥,NaN,6,2024
377398,2024-09-30,A950180,J,NaN,코스닥,NaN,9,2024


In [6]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377400 entries, 0 to 377399
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   date      377400 non-null  datetime64[ns]
 1   firmcode  377400 non-null  object        
 2   ind_code  364400 non-null  object        
 3   asset     227097 non-null  object        
 4   exchange  210827 non-null  object        
 5   ebitda    192348 non-null  object        
 6   month     377400 non-null  int32         
 7   year      377400 non-null  int32         
dtypes: datetime64[ns](1), int32(2), object(5)
memory usage: 20.2+ MB


In [7]:

result["asset_lag"] = result.groupby("firmcode")["asset"].shift(-1)
result["cashflow"] = result["ebitda"] / result["asset_lag"]
print(result.shape)
result = result.dropna(subset=["date"])
print(result.shape)

(377400, 10)
(377400, 10)


In [8]:
result = result[result["month"].isin([3, 6, 9, 12])].sort_values(["firmcode", "date"])
print(result.shape, result["firmcode"].nunique())

result = result[~result["ind_code"].isin(["K", "E", "D"])]
print(result.shape, result["firmcode"].nunique())
# K: 금융업
# D: 전기, 가스, 증기 및 공기 조절 공급업
# E: 수도, 하수, 및 폐기물 처리, 원료 재생업


for k in result.columns:
  print(k, result[k].isna().sum())
  result.loc[result[k].isna(), k] = None

print(result.shape)
result = result.drop_duplicates()
print(result.shape)
result

(373626, 10) 3774
(331650, 10) 3350
date 0
firmcode 0
ind_code 12870
asset 118474
exchange 139225
ebitda 151860
month 0
year 0
asset_lag 120207
cashflow 155151
(331650, 10)
(331650, 10)


,date,firmcode,ind_code,asset,exchange,ebitda,month,year,asset_lag,cashflow
82300,2000-03-31,A000020,C,231378084000,유가증권시장,1458265000,3.0,2000.0,254488830000,0.00573
82301,2000-06-30,A000020,C,254488830000,유가증권시장,12969835000,6.0,2000.0,254488830000,0.050964
82302,2000-09-30,A000020,C,254488830000,유가증권시장,12969835000,9.0,2000.0,254488830000,0.050964
82303,2000-12-31,A000020,C,254488830000,유가증권시장,12969835000,12.0,2000.0,254488830000,0.050964
82304,2001-03-31,A000020,C,254488830000,유가증권시장,12969835000,3.0,2001.0,215747248000,0.060116
...,...,...,...,...,...,...,...,...,...,...
114094,2023-09-30,A950220,M,92151554000,코스닥,-53502626000,9.0,2023.0,92151554000,-0.580594
114095,2023-12-31,A950220,M,92151554000,코스닥,-51550014000,12.0,2023.0,None,None
114096,2024-03-31,A950220,M,None,코스닥,None,3.0,2024.0,None,None
114097,2024-06-30,A950220,M,None,코스닥,None,6.0,2024.0,None,None


In [9]:
result["ind_code"].value_counts(dropna=False)

ind_code
C       209187
J        42966
G        24453
M        21483
None     12870
F         9999
H         3168
N         2574
P         1485
R         1287
I          693
L          594
A          594
B          198
S           99
Name: count, dtype: int64

In [10]:

result_temp = result.dropna(axis=0)

t = result_temp["firmcode"].value_counts()
print(t.shape)
# Delete under 3 years (8 quarters)
# Not consider "continuity" yet (may forever)
temp = pd.merge(t[t < 8], result_temp, on=["firmcode"], how="left")
temp[["firmcode", "month"]].value_counts().sort_index().reset_index().T


(2979,)


,0,1,2,3,4,5,6,7,8,9,...,575,576,577,578,579,580,581,582,583,584
firmcode,A000200,A000200,A000200,A000200,A000310,A000310,A000310,A001690,A001690,A001690,...,A460850,A460850,A460860,A460860,A462520,A465770,A950030,A950030,A950030,A950030
month,3.0,6.0,9.0,12.0,6.0,9.0,12.0,3.0,6.0,9.0,...,6.0,9.0,6.0,9.0,9.0,9.0,3.0,6.0,9.0,12.0
count,1,1,1,1,2,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1,2


In [11]:
result = result[~result["firmcode"].isin(t[t < 8].index)]

In [12]:
result.to_csv("result_temp_3y.csv", index=False)

In [13]:
result = pd.read_csv("result_temp_3y.csv")
result.columns

Index(['date', 'firmcode', 'ind_code', 'asset', 'exchange', 'ebitda', 'month',
       'year', 'asset_lag', 'cashflow'],
      dtype='object')

In [14]:
result[result["month"]==2]["cashflow"].count()

0

In [15]:
print(
    result["firmcode"].nunique(),
    result.shape,
    result["cashflow"].isna().sum(),
    result["cashflow"].isna().sum() / result.shape[0],
    sep="\n"
)

3163
(313137, 10)
138979
0.44382810079933066


In [17]:
result.columns

Index(['date', 'firmcode', 'ind_code', 'asset', 'exchange', 'ebitda', 'month',
       'year', 'asset_lag', 'cashflow'],
      dtype='object')

In [18]:
result["cashflow_num"] = 0.0
result["cashflow_sum"] = 0.0
result["cashflow"] = result["cashflow"].astype(float)
groups = result.reset_index(drop=True).groupby(["firmcode", "month"])


def quater_sum(group):
  group = group.reset_index(drop=True)  
  for i in range(0, group.shape[0]):
    if not group.iloc[i]["cashflow"]:
      continue

    condition = ((group["year"] - group.iloc[i]["year"] >= 0.0) & (group["year"] - group.iloc[i]["year"] < MAX_YEAR))
    # condition &= (group["month"] == group.iloc[i]["month"]) 
    if group.loc[condition].shape[0] == 0:
      continue

    for k, v in {
      "cashflow_sum": group.iloc[i]["cashflow"], 
      "cashflow_num": 1.0
    }.items():
      group.loc[condition, k] = group.loc[condition, k].add(v, fill_value=0).astype(float)
      
  return group.reset_index(drop=True)

result_freq = groups.apply(quater_sum).reset_index(drop=True)
result_freq # 10m

C:\Users\DYK\AppData\Local\Temp\ipykernel_12928\3410614397.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_freq = groups.apply(quater_sum).reset_index(drop=True)


,date,firmcode,ind_code,asset,exchange,ebitda,month,year,asset_lag,cashflow,cashflow_num,cashflow_sum
0,2000-03-31,A000020,C,2.313781e+11,유가증권시장,1.458265e+09,3.0,2000.0,2.544888e+11,0.005730,1.0,0.005730
1,2001-03-31,A000020,C,2.544888e+11,유가증권시장,1.296984e+10,3.0,2001.0,2.157472e+11,0.060116,2.0,0.065846
2,2002-03-31,A000020,C,2.157472e+11,유가증권시장,1.484693e+10,3.0,2002.0,2.014685e+11,0.073694,3.0,0.139540
3,2003-03-31,A000020,C,2.014685e+11,유가증권시장,1.743358e+10,3.0,2003.0,1.942339e+11,0.089756,4.0,0.229295
4,2004-03-31,A000020,C,1.942339e+11,유가증권시장,1.166431e+10,3.0,2004.0,1.952935e+11,0.059727,5.0,0.289022
...,...,...,...,...,...,...,...,...,...,...,...,...
313132,2019-12-31,A950220,M,NaN,NaN,NaN,12.0,2019.0,9.630732e+10,NaN,5.0,0.000000
313133,2020-12-31,A950220,M,9.630732e+10,NaN,-3.011571e+10,12.0,2020.0,1.892655e+11,-0.159119,5.0,-0.159119
313134,2021-12-31,A950220,M,1.892655e+11,코스닥,-4.656234e+10,12.0,2021.0,1.399733e+11,-0.332651,5.0,-0.491770
313135,2022-12-31,A950220,M,1.399733e+11,코스닥,-5.077918e+10,12.0,2022.0,9.215155e+10,-0.551040,5.0,-1.042810


In [19]:
result_freq.to_csv("result_freq1_3y.csv")

In [20]:
result_freq.reset_index(drop=True).shape, result.shape, result_freq.columns

((313137, 12),
 (313137, 12),
 Index(['date', 'firmcode', 'ind_code', 'asset', 'exchange', 'ebitda', 'month',
        'year', 'asset_lag', 'cashflow', 'cashflow_num', 'cashflow_sum'],
       dtype='object'))

In [21]:
result_freq = pd.read_csv("result_freq1_3y.csv").drop("Unnamed: 0", axis=1)
print(result_freq.columns)
print(result_freq["date"].isna().sum())

Index(['date', 'firmcode', 'ind_code', 'asset', 'exchange', 'ebitda', 'month',
       'year', 'asset_lag', 'cashflow', 'cashflow_num', 'cashflow_sum'],
      dtype='object')
0


In [22]:
result_freq["cashflow_num"].value_counts(dropna=False).sort_index()

cashflow_num
1.0     12652
2.0     12652
3.0     12652
4.0     12652
5.0    262529
Name: count, dtype: int64

In [23]:
# cashflow_num에 3이 없는 firmcode 확인
print(result_freq[result_freq["cashflow_num"] == MAX_YEAR]["firmcode"].nunique())
firms = result_freq[result_freq["cashflow_num"] == MAX_YEAR]["firmcode"].unique()
result_freq = result_freq[result_freq["firmcode"].isin(firms)]
result_freq = result_freq[result_freq["cashflow_num"] >= MAX_YEAR - 1]
result_freq["firmcode"].nunique(), result_freq.shape, result_freq["cashflow_sum"].isna().sum()

3163


(3163, (275181, 12), 0)

In [24]:
result_freq["cashflow_num"].value_counts(dropna=False).sort_index()

cashflow_num
4.0     12652
5.0    262529
Name: count, dtype: int64

In [25]:
result_freq["quater"] = result_freq["cashflow_sum"] / result_freq["cashflow_num"]
result_freq["main_quater"] = None
result_freq["quater_num"] = None
result_freq["quater_all"] = None
result_freq["date"] = pd.to_datetime(result_freq["date"])
print(
    result_freq["date"].isna().sum()
)
result_freq.info()

0
<class 'pandas.core.frame.DataFrame'>
Index: 275181 entries, 3 to 313136
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          275181 non-null  datetime64[ns]
 1   firmcode      275181 non-null  object        
 2   ind_code      263871 non-null  object        
 3   asset         186302 non-null  float64       
 4   exchange      176719 non-null  object        
 5   ebitda        160659 non-null  float64       
 6   month         275181 non-null  float64       
 7   year          275181 non-null  float64       
 8   asset_lag     184372 non-null  float64       
 9   cashflow      157602 non-null  float64       
 10  cashflow_num  275181 non-null  float64       
 11  cashflow_sum  275181 non-null  float64       
 12  quater        275181 non-null  float64       
 13  main_quater   0 non-null       object        
 14  quater_num    0 non-null       object        
 15  quater_all    0 non-

In [ ]:
groups = result_freq.reset_index(drop=True).groupby("firmcode")

def main_quater(group):
  group = group.sort_values("date").reset_index(drop=True)
  for i in range(0, group.shape[0]):
    if pd.isna(group.at[i, "date"]):
      continue
    current_date = group.at[i, "date"]
    start_date = current_date - pd.DateOffset(years=2)
    end_date = current_date - pd.DateOffset(years=1)
    subset = group[(group['date'] > start_date) & (group['date'] <= end_date)]
    if subset.empty:
        continue

    quaters = [
      subset[subset['month'] == v]["quater"].values[0] 
      if subset[subset['month'] == v].shape[0] else 0
      for v in (3, 6, 9, 12)]
    quaters_num = (
      subset[subset['month'] == v]["cashflow_num"].values[0] 
      if subset[subset['month'] == v].shape[0] else 0
      for v in (3, 6, 9, 12))
    
    group.at[i, "main_quater"] = np.argmax(np.array(quaters)) 
    group.at[i, "quater_all"] = " ".join(map(str, quaters))
    group.at[i, "quater_num"] = " ".join(map(str, quaters_num))
    
  return group.reset_index(drop=True)

result_freq = groups.apply(main_quater) # 12m
result_freq.loc[~result_freq["main_quater"].isna(), "main_quater"] = result_freq.loc[
  ~result_freq["main_quater"].isna(), "main_quater"] + 1

C:\Users\DYK\AppData\Local\Temp\ipykernel_12928\1490574088.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_freq = groups.apply(main_quater) # 8.44m


In [27]:
result_freq

date firmcode ind_code         asset exchange        ebitda  \
firmcode                                                                        
A000020  0  2003-03-31  A000020        C  2.014685e+11   유가증권시장  1.743358e+10   
         1  2003-06-30  A000020        C  1.942339e+11   유가증권시장  1.138718e+10   
         2  2003-09-30  A000020        C  1.942339e+11   유가증권시장  1.163411e+10   
         3  2003-12-31  A000020        C  1.942339e+11   유가증권시장  1.155413e+10   
         4  2004-03-31  A000020        C  1.942339e+11   유가증권시장  1.166431e+10   
...                ...      ...      ...           ...      ...           ...   
A950220  82 2023-09-30  A950220        M  9.215155e+10      코스닥 -5.350263e+10   
         83 2023-12-31  A950220        M  9.215155e+10      코스닥 -5.155001e+10   
         84 2024-03-31  A950220        M           NaN      코스닥           NaN   
         85 2024-06-30  A950220        M           NaN      코스닥           NaN   
         86 2024-09-30  A950220        M           NaN      코스닥           NaN   

             month    year     asset_lag  cashflow  cashflow_num  \
firmcode                                                           
A000020  0     3.0  2003.0  1.942339e+11  0.089756           4.0   
         1     6.0  2003.0  1.942339e+11  0.058626           4.0   
         2     9.0  2003.0  1.942339e+11  0.059897           4.0   
         3    12.0  2003.0  1.942339e+11  0.059486           4.0   
         4     3.0  2004.0  1.952935e+11  0.059727           5.0   
...            ...     ...           ...       ...           ...   
A950220  82    9.0  2023.0  9.215155e+10 -0.580594           5.0   
         83   12.0  2023.0           NaN       NaN           5.0   
         84    3.0  2024.0           NaN       NaN           5.0   
         85    6.0  2024.0           NaN       NaN           5.0   
         86    9.0  2024.0           NaN       NaN           5.0   

             cashflow_sum    quater main_quater       quater_num  \
firmcode                                                           
A000020  0       0.229295  0.057324        None             None   
         1       0.264939  0.066235        None             None   
         2       0.266211  0.066553        None             None   
         3       0.265799  0.066450        None             None   
         4       0.289022  0.057804           1        4.0 0 0 0   
...                   ...       ...         ...              ...   
A950220  82     -1.527537 -0.305507           4  5.0 5.0 5.0 5.0   
         83     -1.042810 -0.208562           1  5.0 5.0 5.0 5.0   
         84     -1.206674 -0.241335           2  5.0 5.0 5.0 5.0   
         85     -1.206674 -0.241335           3  5.0 5.0 5.0 5.0   
         86     -1.527537 -0.305507           4  5.0 5.0 5.0 5.0   

                                                    quater_all  
firmcode                                                        
A000020  0                                                None  
         1                                                None  
         2                                                None  
         3                                                None  
         4                           0.05732380825690338 0 0 0  
...                                                        ...  
A950220  82  -0.12521600251335555 -0.12521600251335555 -0.1...  
         83  -0.12521600251335555 -0.12521600251335555 -0.1...  
         84  -0.24133477355437347 -0.12521600251335555 -0.1...  
         85  -0.24133477355437347 -0.24133477355437347 -0.1...  
         86  -0.24133477355437347 -0.24133477355437347 -0.3...  

[275181 rows x 16 columns]

In [28]:
result_freq["date"].value_counts().sort_index()

date
2003-03-31    3163
2003-06-30    3163
2003-09-30    3163
2003-12-31    3163
2004-03-31    3163
              ... 
2023-09-30    3163
2023-12-31    3163
2024-03-31    3163
2024-06-30    3163
2024-09-30    3163
Name: count, Length: 87, dtype: int64

In [29]:
result_freq.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 275181 entries, ('A000020', 0) to ('A950220', 86)
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          275181 non-null  datetime64[ns]
 1   firmcode      275181 non-null  object        
 2   ind_code      263871 non-null  object        
 3   asset         186302 non-null  float64       
 4   exchange      176719 non-null  object        
 5   ebitda        160659 non-null  float64       
 6   month         275181 non-null  float64       
 7   year          275181 non-null  float64       
 8   asset_lag     184372 non-null  float64       
 9   cashflow      157602 non-null  float64       
 10  cashflow_num  275181 non-null  float64       
 11  cashflow_sum  275181 non-null  float64       
 12  quater        275181 non-null  float64       
 13  main_quater   262529 non-null  object        
 14  quater_num    262529 non-null  object        


In [30]:
result_freq.rename(columns={"firmcode": "firmcode.1"}).reset_index(drop=False)[["firmcode", "date"]].value_counts(dropna=False).sort_index()

firmcode  date      
A000020   2003-03-31    1
          2003-06-30    1
          2003-09-30    1
          2003-12-31    1
          2004-03-31    1
                       ..
A950220   2023-09-30    1
          2023-12-31    1
          2024-03-31    1
          2024-06-30    1
          2024-09-30    1
Name: count, Length: 275181, dtype: int64

In [31]:
result_freq = result_freq.reset_index(drop=True)
# result_freq["date"] = result_freq["date"].dt.strftime("%Y-%m-%d")
print(result_freq.columns)
print(result_freq["date"].isna().sum())
print(result_freq.shape)
result_freq = result_freq.dropna(subset=["date"])
print(result_freq.shape)
result_freq.to_csv("result_freq_3y.csv", index=False)

Index(['date', 'firmcode', 'ind_code', 'asset', 'exchange', 'ebitda', 'month',
       'year', 'asset_lag', 'cashflow', 'cashflow_num', 'cashflow_sum',
       'quater', 'main_quater', 'quater_num', 'quater_all'],
      dtype='object')
0
(275181, 16)
(275181, 16)


In [32]:
print(result_freq.shape)

(275181, 16)


In [33]:
# group = result_freq[result_freq["firmcode"] == "A000020"]
# i = 5
# current_date = group.iloc[i]["date"]
# start_date = current_date - pd.DateOffset(years=2)
# end_date = current_date - pd.DateOffset(years=1)
# subset = group[(group['date'] > start_date) & (group['date'] <= end_date)]

In [34]:
result_freq = pd.read_csv("result_freq_3y.csv") 

In [35]:
(result_freq["quater_num"].value_counts(dropna=False).sort_index() / result_freq.shape[0]).tail(10)

quater_num
4.0 0 0 0          0.011494
4.0 4.0 0 0        0.011494
4.0 4.0 4.0 0      0.011494
4.0 4.0 4.0 4.0    0.011494
5.0 4.0 4.0 4.0    0.011494
5.0 5.0 4.0 4.0    0.011494
5.0 5.0 5.0 4.0    0.011494
5.0 5.0 5.0 5.0    0.873563
NaN                0.045977
Name: count, dtype: float64

In [40]:
result_freq["quater_num"].value_counts(dropna=False).sort_index().tail(10)

quater_num
4.0 0 0 0            3163
4.0 4.0 0 0          3163
4.0 4.0 4.0 0        3163
4.0 4.0 4.0 4.0      3163
5.0 4.0 4.0 4.0      3163
5.0 5.0 4.0 4.0      3163
5.0 5.0 5.0 4.0      3163
5.0 5.0 5.0 5.0    240388
NaN                 12652
Name: count, dtype: int64

In [36]:
result_freq["quater_num"].value_counts(dropna=False).sort_index().tail(10) / result_freq.dropna(subset=["date"]).shape[0]

quater_num
4.0 0 0 0          0.011494
4.0 4.0 0 0        0.011494
4.0 4.0 4.0 0      0.011494
4.0 4.0 4.0 4.0    0.011494
5.0 4.0 4.0 4.0    0.011494
5.0 5.0 4.0 4.0    0.011494
5.0 5.0 5.0 4.0    0.011494
5.0 5.0 5.0 5.0    0.873563
NaN                0.045977
Name: count, dtype: float64

In [37]:
result_freq["date"].value_counts(dropna=False)

date
2003-03-31    3163
2016-12-31    3163
2018-12-31    3163
2018-09-30    3163
2018-06-30    3163
              ... 
2009-12-31    3163
2009-09-30    3163
2009-06-30    3163
2009-03-31    3163
2024-09-30    3163
Name: count, Length: 87, dtype: int64

In [41]:
import pandas as pd

result_freq = pd.read_csv("result_freq_3y.csv")
result_freq["quaters_num_str"] = result_freq["quater_num"].apply(lambda x: x.split(" ") if isinstance(x, str) else ["0"]*4)
result_freq["quaters_num_cnt"] = result_freq["quater_num"].apply(lambda x: sum(list(map(float, x.split(" ")))) if isinstance(x, str) else 0)

In [47]:
result_freq[result_freq["quaters_num_cnt"] == MAX_YEAR*4].to_csv("result_5y.csv", index=False)

In [48]:
result_freq["quaters_num_cnt"].value_counts().sort_index()

quaters_num_cnt
0.0      12652
4.0       3163
8.0       3163
12.0      3163
16.0      3163
17.0      3163
18.0      3163
19.0      3163
20.0    240388
Name: count, dtype: int64

In [43]:
result_freq.shape[0]

275181

In [44]:
result_freq["quaters_num_cnt"].value_counts().sort_index()/result_freq.shape[0]

quaters_num_cnt
0.0     0.045977
4.0     0.011494
8.0     0.011494
12.0    0.011494
16.0    0.011494
17.0    0.011494
18.0    0.011494
19.0    0.011494
20.0    0.873563
Name: count, dtype: float64

In [45]:
print(result_freq["firmcode"].nunique())

3163
